In [4]:
from bg_atlasapi import BrainGlobeAtlas
# this downloads the atlas and meshes,we could also do it straight from the allen but this is easier.
bg_atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)  

In [5]:
import json
from pathlib import Path
import numpy as np
import pyvista as pv # for plotting 3d
import pandas as pd

from viz_helpers import Probe

In [6]:
MOUSE = 'MMtest'
#insertion_data = pd.read_excel('../data/insertions.xlsx') #this breaks pyvista so have to use csv
#insertion_data.to_csv('../data/insertions.csv', index=False, sep='\t')
#insertion_data = pd.read_csv('../data/insertions.csv', sep='\t')
#insertion_data  = insertion_data[~pd.isna(insertion_data.Mouse)]
#insertion_data = insertion_data[insertion_data.Mouse == MOUSE]

In [7]:
#probes, sides, areas = [],[],[]
#for index, row in insertion_data.iterrows():
#    angles = [row['Elevation'], row['Spin'], row['Azimuth']]
#    mlapdv = row[['ML','AP','DV']].values.astype(int)
#    print(mlapdv)
#    type = row['Probe']
#    print(angles)
#    probes.append(Probe(type, mlapdv, angles))
#    print(angles)
#
#    areaname = row['Targets (Beryl)'].replace(' ','').replace('[','').replace(']','').split(',')
#    areaname = [a.split('-') for a in areaname]
#    areaname, side = list(zip(*areaname))
#    side = ['left' if s == 'lh' else 'right' for s in side]
#    areas.extend(areaname)
#    sides.extend(side)
#
#areas, inds = np.unique(areas, return_index=True)
#areas = areas.tolist()
#areas.append('root')
#sides = np.array(sides)[inds].tolist()

In [8]:
# load the atlas metadata
atlaslocation =  Path('~').expanduser()/'.brainglobe'/'allen_mouse_25um_v1.2/'
with open(atlaslocation/'structures.json','r') as fd:
    structures = json.load(fd)
with open(atlaslocation/'metadata.json','r') as fd:
    metadata = json.load(fd)
structures = pd.DataFrame(structures)

# AP,DV,ML
BREGMA_LOCATION = np.array([216, 18,228 ])*metadata['resolution'] #don't trust too much, for the 25um atlas from https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858
#BREGMA_LOCATION = [540, 44, 570] #don't trust too much, for the 10um atlas from https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858

In [9]:
# load meshes
def load_structure_mesh(atlaslocation,structures,acronym):
    # meshes are in um
    id = structures[structures.acronym == acronym].id.values
    if len(id):
        id = id[0]
    else:
        return
    mesh = atlaslocation/'meshes'/f'{id}.obj'
    mesh = pv.read(mesh)
    return mesh, structures[structures.acronym == acronym].iloc[0]

toplot = []
#for acronym in ['root','CP','LP','VISp','VISa','SCs','ACA','MD']:
for acronym in ['root','MOp','CP','VISp','VISa','SCs','ACA','MD']:
#for acronym in areas:
    toplot.append(load_structure_mesh(atlaslocation,structures,acronym))

In [10]:
#setup the scene
pv.set_jupyter_backend('trame')
p = pv.Plotter(window_size = [1000,1000])# for saving in the next cell with high res
#p = pv.Plotter(window_size = [600,400])

axes = p.add_axes()
axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)
axes.origin = BREGMA_LOCATION #this is the camera and rotation origin, not the true origin of the axes

In [11]:
#add brain regions to scene
rotate5deg = True

for s in toplot:
    s[0].rotate_y(90, point=axes.origin, inplace=True) # rotate the meshes so that [x,y,z] => [ML,AP,DV]
    s[0].rotate_x(-90, point=axes.origin, inplace=True)
    if rotate5deg:
        #FIXME: is the following line positive or negative?
        s[0].rotate_x(-5,point=axes.origin, inplace=True) # allenCCF has a 5 degree tilt
    if s[1].acronym == 'root':
        p.add_mesh(s[0], color = s[1]['rgb_triplet'], opacity = 0.1,silhouette=False)
    else:
        p.add_mesh(s[0],color=s[1]['rgb_triplet'],opacity = 0.7, silhouette=dict(color='#000000',line_width=1) )

In [12]:
# add the probes to the scene
# these are probes for MM016
probes = [Probe('24', np.array([-2037,887,-4542]), np.array([80,0,130])),
          Probe('24', np.array([-1351, 459, -4105]), np.array([70,0,-125])),
          Probe('24', np.array([-3378, -921, -4672]), np.array([75,75,35])),
          Probe('24', np.array([-3148, -2477, -3490]), np.array([60,0,-20])),
          Probe('24', np.array([-1860, -5, -3708]), np.array([45,0,-60]))]
for prb in probes:
    circ = pv.Sphere(radius=100, center=prb.origin + BREGMA_LOCATION)
    p.add_mesh(circ, opacity=1)
    for shnk in prb.shanks:
        rect = pv.Rectangle(shnk.shank_vectors + BREGMA_LOCATION)
        #todo: PLOT A BALL AT THE ORIGIN OF THE PROBE
        p.add_mesh(rect,color='#000000',opacity = 1,line_width=3)
p.show()

Widget(value='<iframe src="http://localhost:55699/index.html?ui=P_0x1ffe13e5ab0_0&reconnect=auto" class="pyvis…